## Create Plots of Streamflow

This notebook creates the plots of streamflow change that pop up as a 'tool tip' when a user scrolls over a gauge site using the results data.

Import packages

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import datetime as dt
import matplotlib as mpl
from datetime import timedelta
from scipy import stats
# sns.set()

fttom = 0.3048

Import gauge results

In [12]:
gage_info = pd.read_csv(r'study_data\gauge_results.csv')

Define an offset date to compute change with respect to:

In [13]:
offset_date = pd.Timestamp('2015-01-01')

Define a function that will compute closest time to offset date:

In [14]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

Load data, make & save plots

In [17]:
for i, gage in gage_info.iterrows():
    gage_id = gage['#']
    gagename = gage['USGS Gagename']
    gageno = gage['USGS Gageno']
    streamflow_data_fn = r'study_data\streamflow_change\%s_%s_return_period_flows.csv'%(gagename, gageno)
    streamflow_data = pd.read_csv(streamflow_data_fn, index_col=1)
    streamflow_data.index = pd.to_datetime(streamflow_data.index)
    streamflow_data = streamflow_data.loc[~streamflow_data.index.duplicated(), :]
    closest_time = nearest(streamflow_data.index, offset_date)
    offset_streamflow = streamflow_data['RP_flows'].loc[closest_time]
    streamflow_data['RP_flows'] -= offset_streamflow

    f, ax = plt.subplots(figsize=(6,2))
    ax.plot(
        streamflow_data.index,
        streamflow_data['RP_flows'],
        c='royalblue',
        marker='o',
    )
    ax.set_ylabel('streamflow \n change '+ r'$(m^3/s)$', fontsize=13)
    ax.set_xlabel('date', fontsize=14)
    ax.set_title('%d: %s (%s)'%(gage_id, gagename, gageno), fontsize=16)
    ax.set_xlim(pd.to_datetime('01-01-1940'), pd.to_datetime('01-01-2020'))
    plt.xticks(rotation=40, fontsize=12)
    plt.yticks(fontsize=12)
    plt.savefig(r'figures\streamflow_change\%s_%s.png'%(gagename, gageno),
                dpi=300,
                bbox_inches='tight',
                transparent=False)
    plt.close()